In [ ]:
from typing import Literal
def _decide_target(left: list[list[float] | None], right: list[list[float] | None]) -> tuple[list[float], Literal["left", "right"] | None] :
            """
            Decides which hand to continue our preprocessing step for.
            
            That is, choosing the longest list when all 'None's are removed.
            """
            left_landmarks = sum([elm for elm in left if elm is not None],[])
            right_landmarks = sum([elm for elm in right if elm is not None],[])
            target = max((left_landmarks, "left"), 
                        (right_landmarks,"right"), 
                        key=lambda x : len(x[0]))
            if not target[0]: #both were full of "None" somehow??
                return [],None
            return target

In [ ]:
import dynamic_signs.csv_reader as csv_reader
import numpy as np

from sign.trajectory import TrajectoryBuilder
import time

reader = csv_reader.csv_reader()
#PATH = "../backend/small_dyn.csv"
PATH = "../backend/new_youtube.csv"
start = time.time()
print("Loading data")
data = reader.extract_two_handed_landmarks(PATH)
end = time.time()
print(f"Loading took {end-start} seconds")

bob = TrajectoryBuilder(target_len=6)
xs = []
ys:list[str]= []
DIM = (-1,21,3)

print("")
for label, videos in data.items():
    for video_id, frames in videos.items():
        left_frames, right_frames = zip(*[frame.get_landmarks() for frame in frames])
        target, hand = _decide_target(left_frames, right_frames) 
        target = np.array(bob.enforce_target_length(list(np.array(target).reshape(DIM)))).flatten()
        xs.append(target)
        ys.append(label + "_" + str(hand))
        #print(f"{label} : {video_id} -> {hand}")

X = np.array(xs, dtype=np.float32)
Y = np.array(ys)
X.shape, len(Y)
        

In [ ]:
import matplotlib.pyplot as plt
from sklearn import decomposition
import mpl_toolkits.mplot3d 

fig = plt.figure(1, figsize=(4,3))
plt.clf()

ax = fig.add_subplot(111, projection="3d", elev=48, azim=134)
ax.set_position([0,0,1,2])#type: ignore


plt.cla()
pca = decomposition.PCA()
pca.fit(X,Y)
Xt = pca.transform(X)

print(len(Xt[0]))
#for name in Y:

name = np.unique(Y)[69]
ax.scatter(Xt[Y == name,1], Xt[Y == name,8])
ax.text(
    0,0,0,
    s=name,
    ha='left',
    va='top',
    transform=ax.transAxes
)

#list(zip(np.unique(Y),range(len(np.unique(Y)))))
# for label, name in enumerate(np.unique(Y)):
    
#     ax.text3D(
#         X[Y == name, 0].mean(),
#         X[Y == name, 1].mean() + 1.5,
#         X[Y == name, 2].mean(),
#         name,
#         horizontalalignment="center",
#         bbox=dict(alpha=0.5, edgecolor="w", facecolor="w"),
#     )
# # Reorder the labels to have colors matching the cluster results
# y = np.choose(Y, [1, 2, 0]).astype(float)
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap=plt.cm.nipy_spectral, edgecolor="k")

# ax.xaxis.set_ticklabels([])
# ax.yaxis.set_ticklabels([])
# ax.zaxis.set_ticklabels([])

# plt.show()  


In [76]:
from sign.training.load_data.dynamic_loader import DynamicLoader

loader = DynamicLoader(target_len=24)
yt_data = loader.prepare_training_data("../backend/youtube_final_j_z.csv")
xs_train, ys_train, xs_test, ys_test = yt_data.spliterino()

f"Len_train: {len(xs_train)}, Len_test: {len(xs_test)}"

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥


'Len_train: 3000, Len_test: 751'

In [81]:
import numpy as np
uniques, counts = np.unique(ys_train, return_counts=True)
unique_filter = uniques[counts > 100]
length_filter = np.array([y for y in ys_train if len(y) > 2 or y in ["J", "Z"]])

chosen_filter = length_filter
indices = np.where(np.isin(ys_train, chosen_filter))
#print(list(zip(uniques,counts)))

X = np.array(xs_train)[indices]
y = np.array(ys_train)[indices]
", ".join(set(y)), f"Len X: {len(X)}"

('Okay, Why, Wow, Yes, J, Again, Ready, Bye, Hello, Z', 'Len X: 1131')

In [82]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
clf = RandomForestClassifier()
#clf = make_pipeline(StandardScaler(), SVC(C=25, coef0=0.5, degree=4))
#clf = LogisticRegression(max_iter=2_000)


cross_validate(clf, X, y)

{'fit_time': array([1.20194387, 1.22490621, 1.21279907, 1.20991588, 1.19858074]),
 'score_time': array([0.0062139 , 0.00568986, 0.00474429, 0.00497961, 0.00499487]),
 'test_score': array([0.72687225, 0.7079646 , 0.75221239, 0.73451327, 0.69469027])}

In [83]:
from sklearn.calibration import cross_val_predict
from sklearn.metrics import classification_report

cv_predictions = cross_val_predict(clf, X, y)
print(classification_report(y, cv_predictions))

              precision    recall  f1-score   support

       Again       0.38      0.28      0.32        40
         Bye       0.36      0.20      0.26        84
       Hello       0.52      0.84      0.64       252
           J       0.97      0.98      0.98       193
        Okay       0.36      0.14      0.20        37
       Ready       0.65      0.53      0.58        49
         Why       0.55      0.31      0.39        59
         Wow       0.38      0.07      0.12        40
         Yes       0.66      0.39      0.49        54
           Z       0.97      0.99      0.98       323

    accuracy                           0.73      1131
   macro avg       0.58      0.47      0.50      1131
weighted avg       0.71      0.73      0.70      1131

